**Hello Everyone**
<br/>

This is a comprehensive tutorial on Developing ensemble models in PyTorch. I will also give a detailed explanation of how to build a model using PyTorch. 
**This notebook has been divided into following sections:**

* Introduction
    1.   Limitations of Tensorflow and Keras
    2.  Advantages of PyTorch


* Custom Data Loading and Preprocessing
    1. Illustrating the APIs
    2. Data Loading and Preprocessing


* Model Training and Evaluation
    1. Defining the model architecture
    2. Defining and Loss function and Optimizer
    3. Training the model
    4. Validating the model


* Data Augmentation
    1. Augmenting more data


* Advanced Guide to Ensembling Models
    1. Defining various models and Training them
    2. Evaluating the models

In [ ]:
import numpy as np
import pandas as pd
import torch.nn.functional as F
import math
from torch.optim import lr_scheduler
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import metrics
import torch
import itertools
from torchvision import models
import torch.optim as optim
from matplotlib.ticker import MaxNLocator
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
from torch.nn import MaxPool2d
import chainer.links as L
from PIL import Image
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
plt.ion()

In [ ]:
#This data is wrongly matched. Please execute this code to have the correct mapping of X and y values

data = np.load('../input/Sign-language-digits-dataset/X.npy')
target = np.load('../input/Sign-language-digits-dataset/Y.npy')
Y = np.zeros(data.shape[0])
Y[:204] = 9
Y[204:409] = 0
Y[409:615] = 7
Y[615:822] = 6
Y[822:1028] = 1
Y[1028:1236] = 8
Y[1236:1443] = 4
Y[1443:1649] = 3
Y[1649:1855] = 2
Y[1855:] = 5
X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size = .02, random_state = 2) ## splitting into train and test set

In [ ]:
#Shapes of X_train and y_train
X_train.shape, y_train.shape

# Custom Data Loading and Processing

Let's understand the work of the following keywords

### torchvision
It is used to load and prepare dataset. Using it you can create *transformations* on the input data.
<br/>
#### transforms
It is used for preprocessing images and performing operations sequentially.
<br/>
#### num_workers
It is used for multiprocessing.Normally, **num_workers = 4 * (number of gpus)** works well.


In [ ]:
class DatasetProcessing(Dataset):
    
    #initialise the class variables - transform, data, target
    def __init__(self, data, target, transform=None): 
        self.transform = transform
        self.data = data.reshape((-1,64,64)).astype(np.float32)[:,:,:,None]
        # converting target to torch.LongTensor dtype
        self.target = torch.from_numpy(target).long() 
    
    #retrieve the X and y index value and return it
    def __getitem__(self, index): 
        return self.transform(self.data[index]), self.target[index]
    
    #returns the length of the data
    def __len__(self): 
        return len(list(self.data))

Data loading in PyTorch can be separated in 2 parts:
<br/>
* Data must be wrapped on a Dataset parent class where the methods __getitem__ and __len__ must be overrided. Not that,  the data is not loaded on memory by now.
* The Dataloader reads the data and puts it into memory.

Here we are working with numpy arrays which can be directly converted to torch variable for mathematical operations.Many datasets contain images which can be imported using PIL library using
<br/>
> *PIL.IMAGE.fromarray(img_name)*

<br/>
We use PIL instead of OpenCV because its Torch default image loader and is compatible with ToTensor() method. The PyTorch website has all the tutorials of this section using their inbuild Dataset class and it can be a toilsome task to load data in case of Custom Dataset.While loading the data, these point should be closely worked upon:
* Setting a batch Size
* Shuffling the data
* Parallelizing the tasks using multiprocecssing workers.
<br/>

The **DataLoader** function  provides all of these features and you can specify them before heading to next task. The label or the target variable can be one an array or one-hot encoded depending on the loss function. For having a look at the different loss functions, please head to [Pytorch loss functions](https://pytorch.org/docs/master/nn.html#loss-functions)

In [ ]:
# preprocessing images and performing operations sequentially
# Firstly, data is converted to PILImage, Secondly, converted to Tensor
# Thirdly, data is Normalized
transform = transforms.Compose(
    [transforms.ToPILImage(), transforms.ToTensor()])


dset_train = DatasetProcessing(X_train, y_train, transform)


train_loader = torch.utils.data.DataLoader(dset_train, batch_size=4,
                                          shuffle=True, num_workers=4)

In [ ]:
dset_test = DatasetProcessing(X_test, y_test, transform)
test_loader = torch.utils.data.DataLoader(dset_test, batch_size=4,
                                          shuffle=True, num_workers=4)

Let's have a look at the Training data

In [ ]:
plt.figure(figsize = (16, 4))
for num, x in enumerate(X_train[0:6]):
    plt.subplot(1,6,num+1)
    plt.axis('off')
    plt.imshow(x)
    plt.title(y_train[num])

# Training

![](https://www.researchgate.net/publication/318174473/figure/fig2/AS:614173574701099@1523441799965/The-topology-of-the-event-driven-Convnet-used-for-this-work-Where-n-is-the-size-of-the.png)








## Define a CNN

Some of the important libraries
### [nn](https://pytorch.org/docs/master/nn.html)
This library contains  function for building conv nets and importing a loss function.
Some loss functions are:
* **binary_cross_entropy**: Function that measures the Binary Cross Entropy between the target and the output.
* **nll_loss**: measures the The negative log likelihood loss.
* **cross_entropy**: This criterion combines log_softmax and nll_loss in a single function.

### [optim](https://pytorch.org/docs/master/optim.html)
torch.optim is a package implementing various optimization algorithms. To construct an Optimizer you have to give it an iterable containing the parameters (all should be Variable s) to optimize. Then, you can specify optimizer-specific options such as the learning rate, weight decay, etc.Some of the optimizers defined by this library are:
* **Adam**: It has been proposed in [Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980)
* **LBFGS**: Implements [L-BFGS algorithm](https://en.wikipedia.org/wiki/Limited-memory_BFGS).
* **RMSprop**: Proposed by G. Hinton in his [course](http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf)
* **SGD**: Nesterov momentum is based on the formula from [On the importance of initialization and momentum in deep learning](http://www.cs.toronto.edu/~hinton/absps/momentum.pdf)
<br/>


Let¡¯s use a **Cross-Entropy loss** and **SGD with momentum**.

In [ ]:
class Net(nn.Module):    
    
    # This constructor will initialize the model architecture
    def __init__(self):
        super(Net, self).__init__()
          
        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            # Putting a 2D Batchnorm after CNN layer
            nn.BatchNorm2d(32),
            # Adding Relu Activation
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
          
        self.linear_layers = nn.Sequential(
            # Adding Dropout
            nn.Dropout(p = 0.5),
            nn.Linear(32 * 32 * 32, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 10),
        )
        
    # Defining the forward pass    
    def forward(self, x):
        
        # Forward Pass through the CNN Layers 
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        # Forwrd pass through Fully Connected Layers
        x = self.linear_layers(x)
        return F.log_softmax(x) 

<!-- background: #5A0000  -->
<!-- color: #fff -->


# How to use optimizers?
To construct an Optimizer you have to give it an iterable containing the parameters (all should be Variable s) to optimize. Then, you can specify optimizer-specific options such as the learning rate, weight decay, etc.<br/>
*If you want to move the model and variables to cuda, use **model_object.cuda()** and **variable_name.cuda()***

**Follow these steps to build your first model in pytorch**


1. Clearing the Gradients of the model parameters 
>       optimizer.zero_grad()

    ** Clear out the gradients accumulated for the parameters of the network before calling loss.backward() and optimizer.step()**

2. Compute the loss
>       criterion( predicted_target, target)

    **Compute the loss between the predicted value and the target value within the loss function previously defined**

3. Backpropogation
>       loss.backward()

    **Compute the updated weights of all the model parameters**

4. Taking an optimization step
>       optimizer.step()

    ** Update the parameters of the network **


In [ ]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [ ]:
########################################
#       Training the model             #
########################################
def train(epoch):
    model.train()
    exp_lr_scheduler.step()
    tr_loss = 0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
            
        # Clearing the Gradients of the model parameters
        optimizer.zero_grad()
        output = model(data)
        pred = torch.max(output.data, 1)[1]
        correct += (pred == target).sum()
        total += len(data)
        
        # Computing the loss
        loss = criterion(output, target)
        
        # Computing the updated weights of all the model parameters
        loss.backward()
        optimizer.step()
        tr_loss = loss.item()
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \t Accuracy: {} %'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.item(),100 * correct / total))
            torch.save(model.state_dict(), './model.pth')
            torch.save(model.state_dict(), './optimizer.pth')
    train_loss.append(tr_loss / len(train_loader))
    train_accuracy.append(100 * correct / total)

## Model Evaluation
Now, it's time to evaluate out model. Here Evaluating means testing the model on the validation set<br/>
*Note that I have used Functional API( F.cross_entropy( args )) so as to give an idea how it works.*

In [ ]:
########################################
#       Evaluating the model           #
########################################

def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    total = 0
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        loss += F.cross_entropy(output, target, size_average=False).item()
        pred = torch.max(output.data, 1)[1]
        total += len(data)
        correct += (pred == target).sum()
    loss /= len(data_loader.dataset)
    valid_loss.append(loss)    
    valid_accuracy.append(100 * correct / total)
    print('\nAverage Validation loss: {:.5f}\tAccuracy: {} %'.format(loss, 100 * correct / total))

### You can run this model for more  epochs

In [ ]:
n_epochs = 50
train_loss = []
train_accuracy = []
valid_loss = []
valid_accuracy = []
for epoch in range(n_epochs):
    train(epoch)
    evaluate(test_loader)

In [ ]:
########################################
#       Plotting the Graph             #
########################################

def plot_graph(epochs):
    fig = plt.figure(figsize=(20,4))
    ax = fig.add_subplot(1, 2, 1)
    plt.title("Train - Validation Loss")
    plt.plot(list(np.arange(epochs) + 1) , train_loss, label='train')
    plt.plot(list(np.arange(epochs) + 1), valid_loss, label='validation')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('loss', fontsize=12)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.legend(loc='best')
    
    ax = fig.add_subplot(1, 2, 2)
    plt.title("Train - Validation Accuracy")
    plt.plot(list(np.arange(epochs) + 1) , train_accuracy, label='train')
    plt.plot(list(np.arange(epochs) + 1), valid_accuracy, label='validation')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('accuracy', fontsize=12)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.legend(loc='best')

In [ ]:
plot_graph(n_epochs)

### Let us now have a look at our model predictions.

In [ ]:
_, (validation_data, target) = next(enumerate(test_loader))
with torch.no_grad():
    output = model(validation_data.cuda())
fig = plt.figure()
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.tight_layout()
    plt.imshow(validation_data[i][0], interpolation='none')
    pred = output.data.max(1, keepdim=True)[1][i].item()
    plt.title("Prediction: {}".format(pred))

### Let us now have a look at the Predicted Probabilities of the predicted words

In [ ]:
with torch.no_grad():
    output = model(validation_data.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())

fig = plt.figure(figsize = (16, 8))
for i in range(0, 4):
    fig.tight_layout()
    plt.style.use('classic')
    plt.subplot(4,2, 2 * i + 1)
    plt.imshow(validation_data[i][0], interpolation='none')
    plt.xticks([])
    plt.yticks([])
    pred = output.data.max(1, keepdim=True)[1][i].item()
    plt.title("Prediction: {}".format(pred))
    plt.subplot(4,2, 2 * i + 2)
    plt.barh([0], [max(prob[i])])
    plt.yticks([])
    plt.title("Predicted Probability: {0:.2f}".format(max(prob[i])))

**One powerful technique to increase the  performance of any deep learning model is through adding more data. So Let's use data augmentation and see if we get some performance improvement.**

## Data Augmentation

In [ ]:
train_transform= transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(), # Horizontal Flip
            transforms.RandomCrop(64, padding=2), # Centre Crop
            transforms.ToTensor(),  #Convereting the input to tensor
            ])
dset_train = DatasetProcessing(X_train, y_train, train_transform)
train_loader = torch.utils.data.DataLoader(dset_train, batch_size=4,
                                          shuffle=True, num_workers=4)

In [ ]:
n_epochs = 50

train_loss = []
train_accuracy = []
valid_loss = []
valid_accuracy = []

for epoch in range(n_epochs):
    train(epoch)
    evaluate(test_loader)

In [ ]:
# Plotting train and validation loss
plot_graph(n_epochs)

# Model Predictions

In [ ]:
_, (validation_data, target) = next(enumerate(test_loader))
with torch.no_grad():
    output = model(validation_data.cuda())
fig = plt.figure()
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.tight_layout()
    plt.imshow(validation_data[i][0], interpolation='none')
    pred = output.data.max(1, keepdim=True)[1][i].item()
    plt.title("Prediction: {}".format(pred))

# Model Predicted Probabilities

In [ ]:
with torch.no_grad():
    output = model(validation_data.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())

fig = plt.figure(figsize = (16, 8))
for i in range(0, 4):
    fig.tight_layout()
    plt.style.use('classic')
    plt.subplot(4,2, 2 * i + 1)
    plt.imshow(validation_data[i][0], interpolation='none')
    plt.xticks([])
    plt.yticks([])
    pred = output.data.max(1, keepdim=True)[1][i].item()
    plt.title("Prediction: {}".format(pred))
    plt.subplot(4,2, 2 * i + 2)
    plt.barh([0], [max(prob[i])])
    plt.yticks([])
    plt.title("Predicted Probability: {0:.2f}".format(max(prob[i])))

# Ensembling Models

**Ensemble methods is a machine learning technique that combines several base models in order to produce one optimal predictive model.  Ensemble methods usually produces more accurate solutions than a single model would. This has been the case in a number of machine learning competitions, where the winning solutions used ensemble methods. **
![](http://images.slideplayer.com/37/10747814/slides/slide_2.jpg)
You can find more detailed and advanced explanation [here](https://www.springer.com/in/book/9781441993250)
<br/>
Finally, it's time to get hands dirty on ensemble. I have also mentioned the layer dims after each conv filter is applied.

In [ ]:
#######################################################################
#       Defining various model architectures for ensembling           #
#######################################################################
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
    
        #1st net
        self.conv1_1 = nn.Conv2d(1, 4, kernel_size=(4, 4), padding = (3, 3), stride=(2, 2)) #34 * 34
        self.conv1_2 = nn.Conv2d(4, 8, kernel_size=(4, 4), padding = (2, 2), stride=(2, 2)) #18 * 18 #mp
        self.fc1_1 = nn.Linear(8 * 9 * 9, 32) #dropout = 0.2
        self.fc1_1_drop = nn.Dropout(p=0.2)
        self.fc1_2 = nn.Linear(32, 10)

        #2nd net
        self.conv2_1 = nn.Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) #64 * 64
        self.conv2_2 = nn.Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) #64 * 64 #mp
        self.conv2_2_drop = nn.Dropout2d()
        self.conv2_3 = nn.Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) #64 * 64
        self.conv2_4 = nn.Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) #64 * 64 #mp
        self.conv2_4_drop = nn.Dropout2d()
        self.fc2_1 = nn.Linear(24 * 16 * 16, 120) #dropout = 0.5
        self.fc2_1_drop = nn.Dropout(p=0.5)
        self.fc2_2 = nn.Linear(120, 10)

        #3rd net
        self.conv3_1 = nn.Conv2d(1, 4, kernel_size=(5, 5), stride=(3, 3), padding=(2, 2)) #22 * 22
        self.conv3_2 = nn.Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) #22 * 22 #mp
        self.conv3_3 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) #11 * 11
        self.fc3_1 = nn.Linear(16 * 11 * 11, 64) #dropout = 0.4
        self.fc3_1_drop = nn.Dropout(p=0.4)
        self.fc3_2 = nn.Linear(64, 10)

    def forward(self, x, y, z):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.pool(self.conv1_2(x)))
        x = x.view(-1, 648) #can also do x.view(-1, 1)
        x = F.relu(self.fc1_1(x))
        x = F.dropout(x, training = self.training)
        x = F.relu(self.fc1_2(x))

        y = F.relu(self.conv2_1(y))
        y = F.relu(self.pool(self.conv2_2_drop(self.conv2_2(y))))
        y = F.relu(self.conv2_3(y))
        y = F.relu(self.pool(self.conv2_4_drop(self.conv2_4(y))))
        y = y.view(-1, 256 * 24)
        y = F.relu(self.fc2_1_drop(self.fc2_1(y)))
        y = F.relu(self.fc2_2(y))

        z = F.relu(self.conv3_1(z))
        z = F.relu(self.pool(self.conv3_2(z)))
        z = F.relu(self.conv3_3(z))
        z = z.view(-1, 16 * 121)
        z = F.relu(self.fc3_1_drop(self.fc3_1(z)))
        z = F.relu(self.fc3_2(z))

        x = torch.cat((x, y, z))

        return F.sigmoid(x)

![](https://saatvikshah1994.github.io/images/ensembling_1/EnsembleDepiction.png)

In [ ]:
########################################
#       Plotting the Graph             #
########################################

def plot_graphs(train_loss, valid_loss, epochs):
    plt.style.use('ggplot')
    fig = plt.figure(figsize=(20,4))
    ax = fig.add_subplot(1, 2, 1)
    plt.title("Train Loss")
    plt.plot(list(np.arange(epochs) + 1) , train_loss, label='train')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('train_loss', fontsize=12)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.legend(loc='best')
    ax = fig.add_subplot(1, 2, 2)
    plt.title("Validation Loss")
    plt.plot(list(np.arange(epochs) + 1), valid_loss, label='test')
    plt.xlabel('num_epochs', fontsize=12)
    plt.ylabel('vaidation _loss', fontsize=12)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.legend(loc='best')

In [ ]:
##################################################
#       Training the ensembled model             #
##################################################

def train_ensemble(epoch):
    model.train()
    exp_lr_scheduler.step()
    tr_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        #print(data.size())
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        target = torch.cat((target, target, target))
        output  = model(data, data, data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        tr_loss += loss.item()
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.item()))
    train_loss.append(tr_loss / len(train_loader))

In [ ]:
####################################################
#       Evaluating the ensembled model             #
####################################################


def evaluate_ensemble(data_loader):
    model.eval()
    loss = 0
    
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        target = torch.cat((target, target, target))
        output  = model(data, data, data)
        # Using the functional API
        loss += F.cross_entropy(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        
    loss /= len(data_loader.dataset)
    valid_loss.append(loss)    
    print('\nAverage Validation loss: {:.5f}\n'.format(loss))

In [ ]:
# Initializing the ensembled model
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

n_epochs = 5

train_loss = []
valid_loss = []

# Training and Evaluating for n_epochs
for epoch in range(n_epochs):
    train_ensemble(epoch)
    evaluate_ensemble(test_loader)

In [ ]:
plot_graphs(train_loss, valid_loss, n_epochs)

So, we learned how to create custom models using PyTorch & also how to impove the accuracy of models. More to come in next versions. **Please Upvote if you liked it.**